# Modeling with SMOTE

In [73]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb

In [74]:
crash_df = pd.read_csv('C:/Users/arshi/OneDrive/Desktop/Soha/STA 141C/Project/Public_Crash_Data_20250529.csv', low_memory = False)

In [75]:
pd.set_option('display.max_columns', 39)
crash_df[crash_df['PRIMARY CONTRIBUTING CIRCUMSTANCE CODE'] == 43.0]

,CRASH DATETIME,DAY OF WEEK CODE,DAY OF WEEK DESCRIPTION,CRASH CLASSIFICATION CODE,CRASH CLASSIFICATION DESCRIPTION,COLLISION ON PRIVATE PROPERTY,PEDESTRIAN INVOLVED,MANNER OF IMPACT CODE,MANNER OF IMPACT DESCRIPTION,ALCOHOL INVOLVED,DRUG INVOLVED,ROAD SURFACE CODE,ROAD SURFACE DESCRIPTION,LIGHTING CONDITION CODE,LIGHTING CONDITION DESCRIPTION,WEATHER 1 CODE,WEATHER 1 DESCRIPTION,WEATHER 2 CODE,WEATHER 2 DESCRIPTION,SEATBELT USED,MOTORCYCLE INVOLVED,MOTORCYCLE HELMET USED,BICYCLED INVOLVED,BICYCLE HELMET USED,LATITUDE,LONGITUDE,PRIMARY CONTRIBUTING CIRCUMSTANCE CODE,PRIMARY CONTRIBUTING CIRCUMSTANCE DESCRIPTION,SCHOOL BUS INVOLVED CODE,SCHOOL BUS INVOLVED DESCRIPTION,WORK ZONE,WORK ZONE LOCATION CODE,WORK ZONE LOCATION DESCRIPTION,WORK ZONE TYPE CODE,WORK ZONE TYPE DESCRIPTION,WORKERS PRESENT,the_geom,COUNTY CODE,COUNTY NAME
14180,05/21/2021 07:00:00 PM +0000,6,Friday,03,Personal Injury Crash,N,N,1.0,Front to rear,N,N,1.0,Dry,1.0,Daylight,1.0,Clear,NaN,NaN,Y,Y,Y,N,N,38.51861,-75.05519,43.0,NaN,0.0,No,N,NaN,NaN,NaN,NaN,N,POINT (-75.055190000068 38.518610000202),S,Sussex


In [79]:
crash_df[crash_df['MANNER OF IMPACT CODE'] == 30.0]

,CRASH DATETIME,DAY OF WEEK CODE,DAY OF WEEK DESCRIPTION,CRASH CLASSIFICATION CODE,CRASH CLASSIFICATION DESCRIPTION,COLLISION ON PRIVATE PROPERTY,PEDESTRIAN INVOLVED,MANNER OF IMPACT CODE,MANNER OF IMPACT DESCRIPTION,ALCOHOL INVOLVED,DRUG INVOLVED,ROAD SURFACE CODE,ROAD SURFACE DESCRIPTION,LIGHTING CONDITION CODE,LIGHTING CONDITION DESCRIPTION,WEATHER 1 CODE,WEATHER 1 DESCRIPTION,WEATHER 2 CODE,WEATHER 2 DESCRIPTION,SEATBELT USED,MOTORCYCLE INVOLVED,MOTORCYCLE HELMET USED,BICYCLED INVOLVED,BICYCLE HELMET USED,LATITUDE,LONGITUDE,PRIMARY CONTRIBUTING CIRCUMSTANCE CODE,PRIMARY CONTRIBUTING CIRCUMSTANCE DESCRIPTION,SCHOOL BUS INVOLVED CODE,SCHOOL BUS INVOLVED DESCRIPTION,WORK ZONE,WORK ZONE LOCATION CODE,WORK ZONE LOCATION DESCRIPTION,WORK ZONE TYPE CODE,WORK ZONE TYPE DESCRIPTION,WORKERS PRESENT,the_geom,COUNTY CODE,COUNTY NAME
380163,05/22/2018 08:52:00 PM +0000,3,Tuesday,02,Property Damage Only,N,N,30.0,NaN,N,N,1.0,Dry,1.0,Daylight,1.0,Clear,NaN,NaN,Y,N,N,N,N,38.7585,-75.6203,18.0,Animal in Roadway - Other Animal,0.0,No,N,NaN,NaN,NaN,NaN,N,POINT (-75.620299999689 38.758500000044),S,Sussex


In [81]:
crash_df = crash_df[(crash_df['CRASH CLASSIFICATION CODE'] != '33') & 
           (crash_df['CRASH CLASSIFICATION CODE'] != '32') & 
           (crash_df['CRASH CLASSIFICATION CODE'] != '31') &
           (crash_df['CRASH CLASSIFICATION CODE'] != '00')]
crash_df['CRASH CLASSIFICATION CODE'] = '0' + crash_df['CRASH CLASSIFICATION CODE'].str[-1]

crash_df = crash_df[crash_df['MANNER OF IMPACT CODE'] != 30.0]

crash_df = crash_df[crash_df['PRIMARY CONTRIBUTING CIRCUMSTANCE CODE'] != 43.0]

crash_df.head()

,CRASH DATETIME,DAY OF WEEK CODE,DAY OF WEEK DESCRIPTION,CRASH CLASSIFICATION CODE,CRASH CLASSIFICATION DESCRIPTION,COLLISION ON PRIVATE PROPERTY,PEDESTRIAN INVOLVED,MANNER OF IMPACT CODE,MANNER OF IMPACT DESCRIPTION,ALCOHOL INVOLVED,DRUG INVOLVED,ROAD SURFACE CODE,ROAD SURFACE DESCRIPTION,LIGHTING CONDITION CODE,LIGHTING CONDITION DESCRIPTION,WEATHER 1 CODE,WEATHER 1 DESCRIPTION,WEATHER 2 CODE,WEATHER 2 DESCRIPTION,SEATBELT USED,MOTORCYCLE INVOLVED,MOTORCYCLE HELMET USED,BICYCLED INVOLVED,BICYCLE HELMET USED,LATITUDE,LONGITUDE,PRIMARY CONTRIBUTING CIRCUMSTANCE CODE,PRIMARY CONTRIBUTING CIRCUMSTANCE DESCRIPTION,SCHOOL BUS INVOLVED CODE,SCHOOL BUS INVOLVED DESCRIPTION,WORK ZONE,WORK ZONE LOCATION CODE,WORK ZONE LOCATION DESCRIPTION,WORK ZONE TYPE CODE,WORK ZONE TYPE DESCRIPTION,WORKERS PRESENT,the_geom,COUNTY CODE,COUNTY NAME
0,11/16/2023 05:10:00 PM +0000,5,Thursday,02,Property Damage Only,N,N,1.0,Front to rear,N,N,1.0,Dry,1.0,Daylight,1.0,Clear,NaN,NaN,Y,N,N,N,N,38.65787,-75.59441,8.0,Following too close,0.0,No,N,NaN,NaN,NaN,NaN,N,POINT (-75.594409999839 38.657869999717),S,Sussex
1,08/21/2022 06:51:00 PM +0000,1,Sunday,03,Personal Injury Crash,N,N,3.0,Angle,N,N,1.0,Dry,1.0,Daylight,2.0,Cloudy,NaN,NaN,Y,N,N,N,N,38.58520,-75.29074,12.0,Driving in a careless or reckless manner,0.0,No,N,NaN,NaN,NaN,NaN,N,POINT (-75.290739999562 38.585200000283),S,Sussex
2,01/19/2018 04:30:00 PM +0000,6,Friday,02,Property Damage Only,N,N,99.0,Unknown,N,N,1.0,Dry,1.0,Daylight,1.0,Clear,NaN,NaN,Y,N,N,N,N,38.53826,-75.05662,99.0,Unknown,0.0,No,N,NaN,NaN,NaN,NaN,N,POINT (-75.056620000321 38.538260000222),S,Sussex
3,06/05/2018 04:30:00 PM +0000,3,Tuesday,01,Non-Reportable,Y,N,4.0,"Sideswipe, same direction",N,N,1.0,Dry,1.0,Daylight,2.0,Cloudy,NaN,NaN,Y,N,N,N,N,39.29256,-75.64020,11.0,"Driver inattention, distraction, or fatigue",0.0,No,N,NaN,NaN,NaN,NaN,N,POINT (-75.640199999988 39.292560000244),K,Kent
4,08/07/2023 01:02:00 PM +0000,2,Monday,01,Non-Reportable,Y,N,NaN,NaN,N,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,N,N,N,N,38.74038,-75.14624,88.0,Other,0.0,No,N,NaN,NaN,NaN,NaN,N,POINT (-75.146239999738 38.740379999925),S,Sussex


In [82]:
percent_missing = (crash_df.isna().sum() / len(crash_df))[(crash_df.isna().sum() / len(crash_df)) > 0]
percent_missing

CRASH DATETIME                                   0.000006
CRASH CLASSIFICATION CODE                        0.000006
CRASH CLASSIFICATION DESCRIPTION                 0.000045
MANNER OF IMPACT CODE                            0.037745
MANNER OF IMPACT DESCRIPTION                     0.037748
ROAD SURFACE CODE                                0.040057
ROAD SURFACE DESCRIPTION                         0.040057
LIGHTING CONDITION CODE                          0.039198
LIGHTING CONDITION DESCRIPTION                   0.039198
WEATHER 1 CODE                                   0.040793
WEATHER 1 DESCRIPTION                            0.040793
WEATHER 2 CODE                                   0.962188
WEATHER 2 DESCRIPTION                            0.962188
PRIMARY CONTRIBUTING CIRCUMSTANCE CODE           0.032318
PRIMARY CONTRIBUTING CIRCUMSTANCE DESCRIPTION    0.032318
SCHOOL BUS INVOLVED CODE                         0.000028
SCHOOL BUS INVOLVED DESCRIPTION                  0.000028
WORK ZONE LOCA

In [84]:
percent_missing = (crash_df.isna().sum() / len(crash_df))[(crash_df.isna().sum() / len(crash_df)) > 0]
percent_missing

CRASH DATETIME                                   0.000006
CRASH CLASSIFICATION CODE                        0.000006
CRASH CLASSIFICATION DESCRIPTION                 0.000045
MANNER OF IMPACT CODE                            0.037745
MANNER OF IMPACT DESCRIPTION                     0.037748
ROAD SURFACE CODE                                0.040057
ROAD SURFACE DESCRIPTION                         0.040057
LIGHTING CONDITION CODE                          0.039198
LIGHTING CONDITION DESCRIPTION                   0.039198
WEATHER 1 CODE                                   0.040793
WEATHER 1 DESCRIPTION                            0.040793
WEATHER 2 CODE                                   0.962188
WEATHER 2 DESCRIPTION                            0.962188
PRIMARY CONTRIBUTING CIRCUMSTANCE CODE           0.032318
PRIMARY CONTRIBUTING CIRCUMSTANCE DESCRIPTION    0.032318
SCHOOL BUS INVOLVED CODE                         0.000028
SCHOOL BUS INVOLVED DESCRIPTION                  0.000028
WORK ZONE LOCA

In [87]:
percent_missing[percent_missing.between(0.01, 0.05)]

MANNER OF IMPACT CODE                            0.037745
MANNER OF IMPACT DESCRIPTION                     0.037748
ROAD SURFACE CODE                                0.040057
ROAD SURFACE DESCRIPTION                         0.040057
LIGHTING CONDITION CODE                          0.039198
LIGHTING CONDITION DESCRIPTION                   0.039198
WEATHER 1 CODE                                   0.040793
WEATHER 1 DESCRIPTION                            0.040793
PRIMARY CONTRIBUTING CIRCUMSTANCE CODE           0.032318
PRIMARY CONTRIBUTING CIRCUMSTANCE DESCRIPTION    0.032318
dtype: float64

In [89]:
percent_missing[percent_missing > 0.05]

WEATHER 2 CODE                    0.962188
WEATHER 2 DESCRIPTION             0.962188
WORK ZONE LOCATION CODE           0.993006
WORK ZONE LOCATION DESCRIPTION    0.993006
WORK ZONE TYPE CODE               0.993002
WORK ZONE TYPE DESCRIPTION        0.993002
dtype: float64

In [91]:
row_drop = list(percent_missing[percent_missing < 0.01].index)
row_drop

['CRASH DATETIME',
 'CRASH CLASSIFICATION CODE',
 'CRASH CLASSIFICATION DESCRIPTION',
 'SCHOOL BUS INVOLVED CODE',
 'SCHOOL BUS INVOLVED DESCRIPTION']

In [93]:
row_fill = list(percent_missing[percent_missing.between(0.01, 0.05)].index)
row_fill

['MANNER OF IMPACT CODE',
 'MANNER OF IMPACT DESCRIPTION',
 'ROAD SURFACE CODE',
 'ROAD SURFACE DESCRIPTION',
 'LIGHTING CONDITION CODE',
 'LIGHTING CONDITION DESCRIPTION',
 'WEATHER 1 CODE',
 'WEATHER 1 DESCRIPTION',
 'PRIMARY CONTRIBUTING CIRCUMSTANCE CODE',
 'PRIMARY CONTRIBUTING CIRCUMSTANCE DESCRIPTION']

In [95]:
col_drop = list(percent_missing[percent_missing > 0.05].index)
col_drop

['WEATHER 2 CODE',
 'WEATHER 2 DESCRIPTION',
 'WORK ZONE LOCATION CODE',
 'WORK ZONE LOCATION DESCRIPTION',
 'WORK ZONE TYPE CODE',
 'WORK ZONE TYPE DESCRIPTION']

In [97]:
crash_df = crash_df.drop(col_drop, axis = 1)
crash_df

,CRASH DATETIME,DAY OF WEEK CODE,DAY OF WEEK DESCRIPTION,CRASH CLASSIFICATION CODE,CRASH CLASSIFICATION DESCRIPTION,COLLISION ON PRIVATE PROPERTY,PEDESTRIAN INVOLVED,MANNER OF IMPACT CODE,MANNER OF IMPACT DESCRIPTION,ALCOHOL INVOLVED,DRUG INVOLVED,ROAD SURFACE CODE,ROAD SURFACE DESCRIPTION,LIGHTING CONDITION CODE,LIGHTING CONDITION DESCRIPTION,WEATHER 1 CODE,WEATHER 1 DESCRIPTION,SEATBELT USED,MOTORCYCLE INVOLVED,MOTORCYCLE HELMET USED,BICYCLED INVOLVED,BICYCLE HELMET USED,LATITUDE,LONGITUDE,PRIMARY CONTRIBUTING CIRCUMSTANCE CODE,PRIMARY CONTRIBUTING CIRCUMSTANCE DESCRIPTION,SCHOOL BUS INVOLVED CODE,SCHOOL BUS INVOLVED DESCRIPTION,WORK ZONE,WORKERS PRESENT,the_geom,COUNTY CODE,COUNTY NAME
0,11/16/2023 05:10:00 PM +0000,5,Thursday,02,Property Damage Only,N,N,1.0,Front to rear,N,N,1.0,Dry,1.0,Daylight,1.0,Clear,Y,N,N,N,N,38.657870,-75.594410,8.0,Following too close,0.0,No,N,N,POINT (-75.594409999839 38.657869999717),S,Sussex
1,08/21/2022 06:51:00 PM +0000,1,Sunday,03,Personal Injury Crash,N,N,3.0,Angle,N,N,1.0,Dry,1.0,Daylight,2.0,Cloudy,Y,N,N,N,N,38.585200,-75.290740,12.0,Driving in a careless or reckless manner,0.0,No,N,N,POINT (-75.290739999562 38.585200000283),S,Sussex
2,01/19/2018 04:30:00 PM +0000,6,Friday,02,Property Damage Only,N,N,99.0,Unknown,N,N,1.0,Dry,1.0,Daylight,1.0,Clear,Y,N,N,N,N,38.538260,-75.056620,99.0,Unknown,0.0,No,N,N,POINT (-75.056620000321 38.538260000222),S,Sussex
3,06/05/2018 04:30:00 PM +0000,3,Tuesday,01,Non-Reportable,Y,N,4.0,"Sideswipe, same direction",N,N,1.0,Dry,1.0,Daylight,2.0,Cloudy,Y,N,N,N,N,39.292560,-75.640200,11.0,"Driver inattention, distraction, or fatigue",0.0,No,N,N,POINT (-75.640199999988 39.292560000244),K,Kent
4,08/07/2023 01:02:00 PM +0000,2,Monday,01,Non-Reportable,Y,N,NaN,NaN,N,N,NaN,NaN,NaN,NaN,NaN,NaN,Y,N,N,N,N,38.740380,-75.146240,88.0,Other,0.0,No,N,N,POINT (-75.146239999738 38.740379999925),S,Sussex
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
536609,09/15/2011 06:44:00 PM +0000,5,Thursday,02,Property Damage Only,N,N,1.0,Front to rear,N,N,1.0,Dry,1.0,Daylight,2.0,Cloudy,Y,N,N,N,N,39.694590,-75.719570,11.0,"Driver inattention, distraction, or fatigue",0.0,No,N,N,POINT (-75.719570000129 39.694590000064),N,New Castle
536610,10/23/2014 12:26:00 PM +0000,5,Thursday,02,Property Damage Only,N,N,1.0,Front to rear,N,N,1.0,Dry,1.0,Daylight,2.0,Cloudy,Y,N,N,N,N,39.604460,-75.748310,11.0,"Driver inattention, distraction, or fatigue",0.0,No,N,N,POINT (-75.74831000044 39.604460000234),N,New Castle
536611,08/23/2009 09:00:00 PM +0000,1,Sunday,02,Property Damage Only,N,N,1.0,Front to rear,N,N,1.0,Dry,1.0,Daylight,1.0,Clear,Y,N,N,N,N,39.628448,-75.745197,NaN,NaN,0.0,No,N,N,POINT (-75.745197099927 39.628447700087),N,New Castle
536612,05/11/2009 02:43:00 PM +0000,2,Monday,02,Property Damage Only,N,N,4.0,"Sideswipe, same direction",N,N,1.0,Dry,1.0,Daylight,2.0,Cloudy,Y,N,N,N,N,39.757961,-75.562487,NaN,NaN,0.0,No,N,N,POINT (-75.562486799556 39.757961200274),N,New Castle


In [99]:
crash_df = crash_df.dropna(subset = row_drop)
crash_df

,CRASH DATETIME,DAY OF WEEK CODE,DAY OF WEEK DESCRIPTION,CRASH CLASSIFICATION CODE,CRASH CLASSIFICATION DESCRIPTION,COLLISION ON PRIVATE PROPERTY,PEDESTRIAN INVOLVED,MANNER OF IMPACT CODE,MANNER OF IMPACT DESCRIPTION,ALCOHOL INVOLVED,DRUG INVOLVED,ROAD SURFACE CODE,ROAD SURFACE DESCRIPTION,LIGHTING CONDITION CODE,LIGHTING CONDITION DESCRIPTION,WEATHER 1 CODE,WEATHER 1 DESCRIPTION,SEATBELT USED,MOTORCYCLE INVOLVED,MOTORCYCLE HELMET USED,BICYCLED INVOLVED,BICYCLE HELMET USED,LATITUDE,LONGITUDE,PRIMARY CONTRIBUTING CIRCUMSTANCE CODE,PRIMARY CONTRIBUTING CIRCUMSTANCE DESCRIPTION,SCHOOL BUS INVOLVED CODE,SCHOOL BUS INVOLVED DESCRIPTION,WORK ZONE,WORKERS PRESENT,the_geom,COUNTY CODE,COUNTY NAME
0,11/16/2023 05:10:00 PM +0000,5,Thursday,02,Property Damage Only,N,N,1.0,Front to rear,N,N,1.0,Dry,1.0,Daylight,1.0,Clear,Y,N,N,N,N,38.657870,-75.594410,8.0,Following too close,0.0,No,N,N,POINT (-75.594409999839 38.657869999717),S,Sussex
1,08/21/2022 06:51:00 PM +0000,1,Sunday,03,Personal Injury Crash,N,N,3.0,Angle,N,N,1.0,Dry,1.0,Daylight,2.0,Cloudy,Y,N,N,N,N,38.585200,-75.290740,12.0,Driving in a careless or reckless manner,0.0,No,N,N,POINT (-75.290739999562 38.585200000283),S,Sussex
2,01/19/2018 04:30:00 PM +0000,6,Friday,02,Property Damage Only,N,N,99.0,Unknown,N,N,1.0,Dry,1.0,Daylight,1.0,Clear,Y,N,N,N,N,38.538260,-75.056620,99.0,Unknown,0.0,No,N,N,POINT (-75.056620000321 38.538260000222),S,Sussex
3,06/05/2018 04:30:00 PM +0000,3,Tuesday,01,Non-Reportable,Y,N,4.0,"Sideswipe, same direction",N,N,1.0,Dry,1.0,Daylight,2.0,Cloudy,Y,N,N,N,N,39.292560,-75.640200,11.0,"Driver inattention, distraction, or fatigue",0.0,No,N,N,POINT (-75.640199999988 39.292560000244),K,Kent
4,08/07/2023 01:02:00 PM +0000,2,Monday,01,Non-Reportable,Y,N,NaN,NaN,N,N,NaN,NaN,NaN,NaN,NaN,NaN,Y,N,N,N,N,38.740380,-75.146240,88.0,Other,0.0,No,N,N,POINT (-75.146239999738 38.740379999925),S,Sussex
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
536609,09/15/2011 06:44:00 PM +0000,5,Thursday,02,Property Damage Only,N,N,1.0,Front to rear,N,N,1.0,Dry,1.0,Daylight,2.0,Cloudy,Y,N,N,N,N,39.694590,-75.719570,11.0,"Driver inattention, distraction, or fatigue",0.0,No,N,N,POINT (-75.719570000129 39.694590000064),N,New Castle
536610,10/23/2014 12:26:00 PM +0000,5,Thursday,02,Property Damage Only,N,N,1.0,Front to rear,N,N,1.0,Dry,1.0,Daylight,2.0,Cloudy,Y,N,N,N,N,39.604460,-75.748310,11.0,"Driver inattention, distraction, or fatigue",0.0,No,N,N,POINT (-75.74831000044 39.604460000234),N,New Castle
536611,08/23/2009 09:00:00 PM +0000,1,Sunday,02,Property Damage Only,N,N,1.0,Front to rear,N,N,1.0,Dry,1.0,Daylight,1.0,Clear,Y,N,N,N,N,39.628448,-75.745197,NaN,NaN,0.0,No,N,N,POINT (-75.745197099927 39.628447700087),N,New Castle
536612,05/11/2009 02:43:00 PM +0000,2,Monday,02,Property Damage Only,N,N,4.0,"Sideswipe, same direction",N,N,1.0,Dry,1.0,Daylight,2.0,Cloudy,Y,N,N,N,N,39.757961,-75.562487,NaN,NaN,0.0,No,N,N,POINT (-75.562486799556 39.757961200274),N,New Castle


In [101]:
for col in row_fill:
    crash_df[col] = crash_df[col].fillna(crash_df[col].mode(dropna = True)[0])
crash_df

C:\Users\arshi\AppData\Local\Temp\ipykernel_41028\701388996.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crash_df[col] = crash_df[col].fillna(crash_df[col].mode(dropna = True)[0])


,CRASH DATETIME,DAY OF WEEK CODE,DAY OF WEEK DESCRIPTION,CRASH CLASSIFICATION CODE,CRASH CLASSIFICATION DESCRIPTION,COLLISION ON PRIVATE PROPERTY,PEDESTRIAN INVOLVED,MANNER OF IMPACT CODE,MANNER OF IMPACT DESCRIPTION,ALCOHOL INVOLVED,DRUG INVOLVED,ROAD SURFACE CODE,ROAD SURFACE DESCRIPTION,LIGHTING CONDITION CODE,LIGHTING CONDITION DESCRIPTION,WEATHER 1 CODE,WEATHER 1 DESCRIPTION,SEATBELT USED,MOTORCYCLE INVOLVED,MOTORCYCLE HELMET USED,BICYCLED INVOLVED,BICYCLE HELMET USED,LATITUDE,LONGITUDE,PRIMARY CONTRIBUTING CIRCUMSTANCE CODE,PRIMARY CONTRIBUTING CIRCUMSTANCE DESCRIPTION,SCHOOL BUS INVOLVED CODE,SCHOOL BUS INVOLVED DESCRIPTION,WORK ZONE,WORKERS PRESENT,the_geom,COUNTY CODE,COUNTY NAME
0,11/16/2023 05:10:00 PM +0000,5,Thursday,02,Property Damage Only,N,N,1.0,Front to rear,N,N,1.0,Dry,1.0,Daylight,1.0,Clear,Y,N,N,N,N,38.657870,-75.594410,8.0,Following too close,0.0,No,N,N,POINT (-75.594409999839 38.657869999717),S,Sussex
1,08/21/2022 06:51:00 PM +0000,1,Sunday,03,Personal Injury Crash,N,N,3.0,Angle,N,N,1.0,Dry,1.0,Daylight,2.0,Cloudy,Y,N,N,N,N,38.585200,-75.290740,12.0,Driving in a careless or reckless manner,0.0,No,N,N,POINT (-75.290739999562 38.585200000283),S,Sussex
2,01/19/2018 04:30:00 PM +0000,6,Friday,02,Property Damage Only,N,N,99.0,Unknown,N,N,1.0,Dry,1.0,Daylight,1.0,Clear,Y,N,N,N,N,38.538260,-75.056620,99.0,Unknown,0.0,No,N,N,POINT (-75.056620000321 38.538260000222),S,Sussex
3,06/05/2018 04:30:00 PM +0000,3,Tuesday,01,Non-Reportable,Y,N,4.0,"Sideswipe, same direction",N,N,1.0,Dry,1.0,Daylight,2.0,Cloudy,Y,N,N,N,N,39.292560,-75.640200,11.0,"Driver inattention, distraction, or fatigue",0.0,No,N,N,POINT (-75.640199999988 39.292560000244),K,Kent
4,08/07/2023 01:02:00 PM +0000,2,Monday,01,Non-Reportable,Y,N,1.0,Front to rear,N,N,1.0,Dry,1.0,Daylight,1.0,Clear,Y,N,N,N,N,38.740380,-75.146240,88.0,Other,0.0,No,N,N,POINT (-75.146239999738 38.740379999925),S,Sussex
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
536609,09/15/2011 06:44:00 PM +0000,5,Thursday,02,Property Damage Only,N,N,1.0,Front to rear,N,N,1.0,Dry,1.0,Daylight,2.0,Cloudy,Y,N,N,N,N,39.694590,-75.719570,11.0,"Driver inattention, distraction, or fatigue",0.0,No,N,N,POINT (-75.719570000129 39.694590000064),N,New Castle
536610,10/23/2014 12:26:00 PM +0000,5,Thursday,02,Property Damage Only,N,N,1.0,Front to rear,N,N,1.0,Dry,1.0,Daylight,2.0,Cloudy,Y,N,N,N,N,39.604460,-75.748310,11.0,"Driver inattention, distraction, or fatigue",0.0,No,N,N,POINT (-75.74831000044 39.604460000234),N,New Castle
536611,08/23/2009 09:00:00 PM +0000,1,Sunday,02,Property Damage Only,N,N,1.0,Front to rear,N,N,1.0,Dry,1.0,Daylight,1.0,Clear,Y,N,N,N,N,39.628448,-75.745197,11.0,"Driver inattention, distraction, or fatigue",0.0,No,N,N,POINT (-75.745197099927 39.628447700087),N,New Castle
536612,05/11/2009 02:43:00 PM +0000,2,Monday,02,Property Damage Only,N,N,4.0,"Sideswipe, same direction",N,N,1.0,Dry,1.0,Daylight,2.0,Cloudy,Y,N,N,N,N,39.757961,-75.562487,11.0,"Driver inattention, distraction, or fatigue",0.0,No,N,N,POINT (-75.562486799556 39.757961200274),N,New Castle


In [103]:
crash_df.isna().sum()

CRASH DATETIME                                   0
DAY OF WEEK CODE                                 0
DAY OF WEEK DESCRIPTION                          0
CRASH CLASSIFICATION CODE                        0
CRASH CLASSIFICATION DESCRIPTION                 0
COLLISION ON PRIVATE PROPERTY                    0
PEDESTRIAN INVOLVED                              0
MANNER OF IMPACT CODE                            0
MANNER OF IMPACT DESCRIPTION                     0
ALCOHOL INVOLVED                                 0
DRUG INVOLVED                                    0
ROAD SURFACE CODE                                0
ROAD SURFACE DESCRIPTION                         0
LIGHTING CONDITION CODE                          0
LIGHTING CONDITION DESCRIPTION                   0
WEATHER 1 CODE                                   0
WEATHER 1 DESCRIPTION                            0
SEATBELT USED                                    0
MOTORCYCLE INVOLVED                              0
MOTORCYCLE HELMET USED         

In [105]:
crash_df.to_csv('clean_crash_data.csv', index = False)

In [107]:
clean_df = pd.read_csv('clean_crash_data.csv')

In [109]:
# dropping the non numeric columns
non_numeric = ['CRASH CLASSIFICATION DESCRIPTION', 'CRASH DATETIME', 'COUNTY NAME']
X = clean_df.drop(columns=non_numeric, errors='ignore')

In [111]:
# identifying high cardinality columns
high_card = X.nunique().sort_values(ascending=False)
print(high_card.head(15))

the_geom                                         467943
LATITUDE                                         114191
LONGITUDE                                         78973
PRIMARY CONTRIBUTING CIRCUMSTANCE CODE               23
PRIMARY CONTRIBUTING CIRCUMSTANCE DESCRIPTION        23
WEATHER 1 DESCRIPTION                                11
WEATHER 1 CODE                                       11
ROAD SURFACE DESCRIPTION                             11
ROAD SURFACE CODE                                    11
MANNER OF IMPACT DESCRIPTION                         10
MANNER OF IMPACT CODE                                10
LIGHTING CONDITION CODE                               8
LIGHTING CONDITION DESCRIPTION                        8
DAY OF WEEK DESCRIPTION                               7
DAY OF WEEK CODE                                      7
dtype: int64


In [113]:
# drop the_geom
X = X.drop(columns=['the_geom'])

In [115]:
# Drop the raw coordinates
X = clean_df.drop(columns=['the_geom', 'LATITUDE', 'LONGITUDE'], errors='ignore')

In [117]:
non_numeric_cols = X.select_dtypes(include=['object', 'datetime64']).columns
X = X.drop(columns=non_numeric_cols)

In [119]:
# identify object columns that would fail standardization
print(X.select_dtypes(include=['object', 'datetime64']).columns)

Index([], dtype='object')


In [121]:
df = clean_df

In [123]:
categorical_cols = [col for col in df.columns if col[-4:] != 'CODE' and col not in ['CRASH DATETIME', 'LATITUDE', 'LONGITUDE', 'the_geom', 'CRASH CLASSIFICATION DESCRIPTION', 'hour', 'dayofweek', 'month', 'year', 'LOCATION_CLUSTER']]
categorical_cols

['DAY OF WEEK DESCRIPTION',
 'COLLISION ON PRIVATE PROPERTY',
 'PEDESTRIAN INVOLVED',
 'MANNER OF IMPACT DESCRIPTION',
 'ALCOHOL INVOLVED',
 'DRUG INVOLVED',
 'ROAD SURFACE DESCRIPTION',
 'LIGHTING CONDITION DESCRIPTION',
 'WEATHER 1 DESCRIPTION',
 'SEATBELT USED',
 'MOTORCYCLE INVOLVED',
 'MOTORCYCLE HELMET USED',
 'BICYCLED INVOLVED',
 'BICYCLE HELMET USED',
 'PRIMARY CONTRIBUTING CIRCUMSTANCE DESCRIPTION',
 'SCHOOL BUS INVOLVED DESCRIPTION',
 'WORK ZONE',
 'WORKERS PRESENT',
 'COUNTY NAME']

In [125]:
categorical_cols = [col for col in df.columns if col[-4:] != 'CODE' and col not in ['CRASH DATETIME', 'LATITUDE', 'LONGITUDE', 'the_geom', 'CRASH CLASSIFICATION DESCRIPTION', 'hour', 'dayofweek', 'month', 'year', 'COUNTY_NAME']]
categorical_cols

['DAY OF WEEK DESCRIPTION',
 'COLLISION ON PRIVATE PROPERTY',
 'PEDESTRIAN INVOLVED',
 'MANNER OF IMPACT DESCRIPTION',
 'ALCOHOL INVOLVED',
 'DRUG INVOLVED',
 'ROAD SURFACE DESCRIPTION',
 'LIGHTING CONDITION DESCRIPTION',
 'WEATHER 1 DESCRIPTION',
 'SEATBELT USED',
 'MOTORCYCLE INVOLVED',
 'MOTORCYCLE HELMET USED',
 'BICYCLED INVOLVED',
 'BICYCLE HELMET USED',
 'PRIMARY CONTRIBUTING CIRCUMSTANCE DESCRIPTION',
 'SCHOOL BUS INVOLVED DESCRIPTION',
 'WORK ZONE',
 'WORKERS PRESENT',
 'COUNTY NAME']

In [127]:
X = df[categorical_cols]
y = df['CRASH CLASSIFICATION DESCRIPTION']

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y= le.fit_transform(y)

encoder = ColumnTransformer([
    ('onehot', OneHotEncoder(), categorical_cols)
])

In [129]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.pipeline import Pipeline
import numpy as np

# Step 1: OneHotEncode categorical features
encoder = ColumnTransformer(
    transformers=[('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_cols)],
    remainder='drop'
)

# Apply encoding to full feature set
X_encoded = encoder.fit_transform(X)

# Step 2: Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y, test_size=0.2, random_state=42, stratify=y
)

# Step 3: Apply SMOTE to the training set only
sm = SMOTE(random_state=42)
X_resampled, y_resampled = sm.fit_resample(X_train, y_train)

# Step 4: Scale the data
scaler = StandardScaler()
X_resampled_scaled = scaler.fit_transform(X_resampled)
X_test_scaled = scaler.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

models = {
    'XGBoost': xgb.XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='mlogloss'),
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000, solver='liblinear'),
    'Random Forest': RandomForestClassifier(random_state=42, n_estimators=100),
    'KNN': KNeighborsClassifier(n_neighbors=5)
}

cmap_dict = {
    'XGBoost': plt.cm.Blues,
    'Logistic Regression': plt.cm.Purples,
    'Random Forest': plt.cm.Greens,
    'KNN': plt.cm.PuBuGn
}

model_performance = {}

for name, model in models.items():
    print(f"\n--- {name} ---")
    model.fit(X_resampled_scaled, y_resampled)
    y_pred = model.predict(X_test_scaled)
    acc = accuracy_score(y_test, y_pred)
    model_performance[name] = acc
    print(f"Accuracy: {acc:.4f}")
    print("Classification Report:\n", classification_report(y_test, y_pred, zero_division=0))

    cm = confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot(cmap=cmap_dict.get(name, plt.cm.viridis), xticks_rotation=45)
    plt.title(f"{name} - Confusion Matrix")
    plt.tight_layout()
    plt.show()

# Summary
pd.DataFrame.from_dict(model_performance, orient='index', columns=['Accuracy']).sort_values(by='Accuracy', ascending=False)